<a href="https://colab.research.google.com/github/marika-rago/Progetto-ML/blob/main/realizzazione_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import librosa
import soundfile as sf
import numpy as np
from pydub import AudioSegment
import matplotlib.pyplot as plt
import random

In [ ]:
input_path = "/content/drive/MyDrive/musdb18/train/" # da modificare
output_path = "/content/drive/MyDrive/musdb18_reduced/" # da modificare

os.makedirs(output_path, exist_ok=True)
hq_dir = os.path.join(output_path, "HQ")
os.makedirs(hq_dir, exist_ok=True)

In [ ]:
# numero di brani da selezionare
n_files = 10   # cambia questo numero a piacere

# lista di tutti i file .wav
all_files = [f for f in os.listdir(input_path) if f.endswith(".wav")]

# settiamo un seed per la riproducibilità
random.seed(42)

# selezione random di n_files (senza ripetizioni)
files = random.sample(all_files, n_files)

print(f"Brani selezionati ({n_files}):")
for f in files:
    print("-", f)

In [ ]:
clip_duration = 10  # secondi
n_clips_per_track = 3  # quante clip estrarre per ogni brano

for f in files:
    file_path = os.path.join(input_path, f) # percorso completo file audio
    y, sr = librosa.load(file_path, sr=None)
    # y è il waveform (array NumPy) normalizzato in [-1, 1]
    # sr è il sample rate originale del file

    total_duration = librosa.get_duration(y=y, sr=sr)

    for i in range(n_clips_per_track):
        # scegle inizio casuale evitando di uscire dalla fine
        max_start = total_duration - clip_duration
        if max_start <= 0:
            # se il brano è più corto di 10s, salta
            continue

        # Converte l’istante di inizio in intero e calcola la fine
        start_time = random.uniform(0, max_start)
        start = int(start_time * sr)
        end = start + clip_duration * sr
        clip = y[start:end] # Estre dal waveform y la finestra corrispondente alla clip

        out_file = os.path.join(hq_dir, f"{f.replace('.wav','')}_clip{i}.wav")
        # Prepara il nome file di uscita dentro la cartella HQ, con suffisso _clip{i}

        sf.write(out_file, clip, sr) # Salva la clip su disco in formato WAV